In [1]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

CUDA = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

batch_size = 256

# 需要將輸入轉換為tensor,需加入transforms.ToTensor()

train_transform = transforms.Compose([
                  # transforms.ColorJitter(brightness=(0, 2), contrast=(0, 2), saturation=(0, 2), hue=(-0.1, 0.1)),
                  transforms.RandomResizedCrop(size = (256,256),scale=(0.7, 1.0), ratio=(1.0, 1.0)),
                  # transforms.RandomVerticalFlip(p = 0.5),
                  transforms.RandomHorizontalFlip(p = 0.5),
                  transforms.Resize((256, 256)),
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
                  transforms.Resize((256, 256)),
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 使用 torchvision.datasets.ImageFolder 讀取訓練資料
image_folder = ImageFolder('D:/project/dataset/AnimalFaces/afhq/train', transform = train_transform, target_transform=None)
# 建立 DataLoader，shuffle 為 True 表示會將資料進行打亂
train_loader = DataLoader(dataset = image_folder, batch_size = batch_size, shuffle = True, num_workers = 2)

# 使用 torchvision.datasets.ImageFolder 讀取測試資料
val_image_folder = ImageFolder('D:/project/dataset/AnimalFaces/afhq/val', transform = val_transform, target_transform=None)
# 建立 DataLoader，shuffle 為 False 表示不會將資料進行打亂
val_loader = DataLoader(dataset = val_image_folder, batch_size = batch_size, shuffle = False, num_workers = 2)


C:\Users\User\Anaconda3\envs\pytorch_py36\lib\site-packages\numpy\__init__.py:138: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init


cuda


In [2]:
class Block(nn.Module):
    def __init__(self, channels):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=channels//4, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels//4)
        self.conv2 = nn.Conv2d(in_channels=channels//4, out_channels=channels//4, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels//4)
        self.conv3 = nn.Conv2d(in_channels=channels//4, out_channels=channels, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(channels)

    def forward(self, x):
        x_residual = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = x_residual + x
        x = F.relu(x)
        return x

In [3]:
class Model(nn.Module):
    def __init__(self, Block):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, padding=0, stride=4)
        self.bn1 = nn.BatchNorm2d(64)
        self.block1 = Block(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2, padding=0, stride=2)
        self.bn2 = nn.BatchNorm2d(128)
        self.block2 = Block(128)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=2, padding=0, stride=2)
        self.bn3 = nn.BatchNorm2d(256)
        self.block3 = Block(256)
        self.drop = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(in_features=256, out_features=3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.block1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.block2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.block3(x)
        
        x = F.max_pool2d(x, kernel_size=x.size()[2:])
        x = torch.flatten(x, 1)
        x = self.drop(x)
        x = self.fc1(x)
        return x

model = Model(Block)
# 若報錯 Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same
# 需要將model轉換為使用GPU
model = model.cuda()

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)


In [ ]:
for epoch in range(10):
    train_loss = 0.0
    train_acc = 0.0
    train_step_count = 0.0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # data, target = Variable(data), Variable(target)
        
        if CUDA:
            data, target = data.cuda(), target.cuda()
            
        # data = data.to(device)
        # target = target.to(device)

        # clear gradient
        optimizer.zero_grad()

        # Forward propagation
        output = model(data)
        loss = criterion(output, target)

        # Calculate gradients
        loss.backward()

        # Update parameters
        optimizer.step()

        predicted = torch.max(output.data, 1)[1]
        train_loss += loss.item()
        train_acc += torch.sum(predicted == target, dtype = torch.float32).item()/batch_size
        train_step_count += 1.0
        
    print('train epoch:', epoch, 'loss', train_loss/train_step_count, 'acc', train_acc/train_step_count)
            
    val_loss = 0.0
    val_acc = 0.0
    val_step_count = 0.0
    model.eval()
    with torch.no_grad():
        for val_batch_idx, (val_data, val_target) in enumerate(val_loader):
            if CUDA:
                val_data, val_target = val_data.cuda(), val_target.cuda()
            val_output = model(val_data)
            val_predicted = torch.max(val_output.data, 1)[1]
            val_loss += criterion(val_output, val_target).item()
            val_acc += torch.sum(val_predicted == val_target, dtype = torch.float32).item()/batch_size
            val_step_count += 1.0
        
    print('validation:', 'loss', val_loss/val_step_count, 'acc', val_acc/val_step_count)
        

train epoch: 0 loss 7.563825932042352 acc 0.35614224137931033
validation: loss 1.0862312018871307 acc 0.3561197916666667
train epoch: 1 loss 1.0825009428221604 acc 0.3838900862068966
validation: loss 1.0679274598757427 acc 0.4088541666666667
train epoch: 2 loss 1.0726471769398656 acc 0.38894127155172414
validation: loss 1.0600715080897014 acc 0.44140625
train epoch: 3 loss 1.0725165872738278 acc 0.3863820043103448
validation: loss 1.051840215921402 acc 0.4453125
train epoch: 4 loss 1.0667609017470787 acc 0.3909617456896552
validation: loss 1.0585270126660664 acc 0.39453125


In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')


In [ ]:
# 讀取權重
model.load_state_dict(torch.load('model_weights.pth'))

train_acc = 0.0
train_step_count = 0.0
model.eval()
with torch.no_grad():
    for train_batch_idx, (train_data, train_target) in enumerate(train_loader):
        if CUDA:
            train_data, train_target = train_data.cuda(), train_target.cuda()
        train_output = model(train_data)
        train_predicted = torch.max(train_output.data, 1)[1]
        train_acc += torch.sum(train_predicted == train_target, dtype = torch.float32).item()/batch_size
        train_step_count += 1.0

print('train:', 'acc', train_acc/train_step_count)

val_acc = 0.0
val_step_count = 0.0
model.eval()
with torch.no_grad():
    for val_batch_idx, (val_data, val_target) in enumerate(val_loader):
        if CUDA:
            val_data, val_target = val_data.cuda(), val_target.cuda()
        val_output = model(val_data)
        val_predicted = torch.max(val_output.data, 1)[1]
        val_acc += torch.sum(val_predicted == val_target, dtype = torch.float32).item()/batch_size
        val_step_count += 1.0

print('validation:', 'acc', val_acc/val_step_count)

train: acc 0.9824728260869565


In [5]:
# 儲存整個模型
torch.save(model, 'model.pth')


C:\Users\User\Anaconda3\envs\pytorch_py36\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
